In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![](https://warehouse-camo.ingress.cmh1.psfhosted.org/072e5a48a57a57f8c239cd3fb717f1c9def7f2bd/68747470733a2f2f6576616c6d6c2d7765622d696d616765732e73332e616d617a6f6e6177732e636f6d2f6576616c6d6c5f686f72697a6f6e74616c2e737667)pypi.org

#EvalML 

https://github.com/alteryx/evalml

EvalML is an AutoML library which builds, optimizes, and evaluates machine learning pipelines using domain-specific objective functions.

Key Functionality

Automation - Makes machine learning easier. Avoid training and tuning models by hand. Includes data quality checks, cross-validation and more.

Data Checks - Catches and warns of problems with your data and problem setup before modeling.

End-to-end - Constructs and optimizes pipelines that include state-of-the-art preprocessing, feature engineering, feature selection, and a variety of modeling techniques.

Model Understanding - Provides tools to understand and introspect on models, to learn how they'll behave in your problem domain.

Domain-specific - Includes repository of domain-specific objective functions and an interface to define your own.

Follow along by cloning the demo repository: https://github.com/alteryx/open_sourc...

For more information on EvalML, visit our GitHub page at https://github.com/alteryx/evalml and our documentation at https://evalml.alteryx.com

https://github.com/alteryx/evalml

In [ ]:
df=pd.read_csv('/kaggle/input/cusersmarildownloadsgermancsv/german.csv',encoding ='ISO-8859-1',sep=";")
df.head()

#Code by Homayoon Khadivi https://www.kaggle.com/homayoonkhadivi/creative-automation-ml-evalml-model-understanding/comments

In [ ]:
!pip install evalml

In [ ]:
X = df.drop(columns=['Creditability'])
y = df['Creditability']

In [ ]:
import evalml
from evalml.pipelines import BinaryClassificationPipeline

#X,y = evalml.demos.load_breast_cancer()

pipeline = BinaryClassificationPipeline(["Simple Imputer", "Random Forest Classifier"])
pipeline.fit(X,y)

print(pipeline.score(X,y, objectives = ['log loss binary']))

#Feature Importance

We can get the importance associated with each feature of the resulting pipeline

In [ ]:
pipeline.feature_importance

#Create a bar plot of the feature importances

In [ ]:
pipeline.graph_feature_importance()

#Permutation Importance

Compute and plot the permutation importance of the pipeline.

In [ ]:
from evalml.model_understanding import calculate_permutation_importance
calculate_permutation_importance(pipeline, X, y, "log loss binary")

In [ ]:
from evalml.model_understanding import graph_permutation_importance
graph_permutation_importance(pipeline, X, y, 'log loss binary')

#Partial Dependence Plots

Calculate the one-way partial dependence plots for a feature. Here, I chose Credit amount.

In [ ]:
from evalml.model_understanding.graphs import partial_dependence
partial_dependence(pipeline, X, features='Credit_Amount')

In [ ]:
from evalml.model_understanding.graphs import graph_partial_dependence
graph_partial_dependence(pipeline, X, features='Credit_Amount')

#Compute the partial dependence for a categorical feature. We don't have categorical, hence I used Demos Load fraud, which has nothing to do with my Creditability Dataset. I saved that snippet to apply when I have Categorical features. 

In [ ]:
X_fraud, y_fraud = evalml.demos.load_fraud(100, verbose=False)
X_fraud.ww.init(logical_types={"provider": "Categorical", 'region': "Categorical"})
fraud_pipeline = BinaryClassificationPipeline(["DateTime Featurization Component","One Hot Encoder", "Random Forest Classifier"])
fraud_pipeline.fit(X_fraud, y_fraud)

graph_partial_dependence(fraud_pipeline, X_fraud, features='provider')

#Two-way partial dependence plots are also possible and invoke the same API.

In [ ]:
partial_dependence(pipeline, X, features=('Account_Balance', 'Duration_of_Credit_monthly'), grid_resolution=10)

In [ ]:
graph_partial_dependence(pipeline, X, features=('Account_Balance', 'Duration_of_Credit_monthly'), grid_resolution=10)

#Confusion Matrix

For binary or multiclass classification, we can view a confusion matrix of the classifier’s predictions. In the DataFrame output of confusion_matrix(), the column header represents the predicted labels while row header represents the actual labels.

In [ ]:
from evalml.model_understanding.graphs import confusion_matrix
y_pred = pipeline.predict(X)
confusion_matrix(y, y_pred)

In [ ]:
from evalml.model_understanding.graphs import graph_confusion_matrix
y_pred = pipeline.predict(X)
graph_confusion_matrix(y, y_pred)

#Predicted Vs Actual Values Graph for Regression Problems

Create a scatterplot comparing predicted vs actual values for regression problems. Specify an outlier_threshold to color values differently if the absolute difference between the actual and predicted values are outside of a given threshold.

In [ ]:
from evalml.model_understanding.graphs import graph_prediction_vs_actual
from evalml.pipelines import RegressionPipeline

#X_regress, y_regress = evalml.demos.load_diabetes()
X_train, X_test, y_train, y_test = evalml.preprocessing.split_data(X, y, problem_type='regression')

pipeline_regress = RegressionPipeline(['One Hot Encoder', 'Linear Regressor'])
pipeline_regress.fit(X_train, y_train)

y_pred = pipeline_regress.predict(X_test)
graph_prediction_vs_actual(y_test, y_pred, outlier_threshold=50)

In [ ]:
pipeline_dt = BinaryClassificationPipeline(['Simple Imputer', 'Decision Tree Classifier'])
pipeline_dt.fit(X, y)

#Tree Visualization

Visualize the structure of the Decision Tree that was fit to that data, and save it if necessary.

In [ ]:
from evalml.model_understanding.graphs import visualize_decision_tree

visualize_decision_tree(pipeline_dt.estimator, max_depth=2, rotate=False, filled=True, filepath=None)

#Why the parenthesis above are empty?

#Explaining Predictions

Explain why the model made certain predictions with the explain_predictions function. This will use the Shapley Additive Explanations (SHAP) algorithm to identify the top features that explain the predicted value.

This function can explain both classification and regression models - all you need to do is provide the pipeline, the input features, and a list of rows corresponding to the indices of the input features you want to explain. The function will return a table that you can print summarizing the top 3 most positive and negative contributing features to the predicted value.

In [ ]:
from evalml.model_understanding.prediction_explanations import explain_predictions

table = explain_predictions(pipeline=pipeline, input_features=X, y=None, indices_to_explain=[3],
                           top_k_features=6, include_shap_values=True)
print(table)

#The interpretation of the table

The interpretation of the table is the same for regression problems - but the SHAP value now corresponds to the change in the estimated value of the dependent variable rather than a change in probability. For multiclass classification problems, a table will be output for each possible class.

Below is an example of how you would explain three predictions with explain_predictions.

In [ ]:
from evalml.model_understanding.prediction_explanations import explain_predictions

report = explain_predictions(pipeline=pipeline, input_features=X, y=y, indices_to_explain=[0, 4, 9], include_shap_values=True,
                            output_format='text')
print(report)

#Explaining Best and Worst Predictions

When debugging machine learning models, it is often useful to analyze the best and worst predictions the model made. The explain_predictions_best_worst function can help us with this.

This function will display the output of explain_predictions for the best 2 and worst 2 predictions. By default, the best and worst predictions are determined by the absolute error for regression problems and cross entropy for classification problems.

We can specify our own ranking function by passing in a function to the metric parameter. This function will be called on y_true and y_pred. By convention, lower scores are better.

At the top of each table, we can see the predicted probabilities, target value, error, and row index for that prediction. For a regression problem, we would see the predicted value instead of predicted probabilities.

In [ ]:
from evalml.model_understanding.prediction_explanations import explain_predictions_best_worst

report = explain_predictions_best_worst(pipeline=pipeline, input_features=X, y_true=y,
                                        include_shap_values=True, top_k_features=6, num_to_explain=2)

print(report)

#Hinge Loss

Use a custom metric (hinge loss) for selecting the best and worst predictions. See this example:

In [ ]:
import numpy as np

def hinge_loss(y_true, y_pred_proba):

    probabilities = np.clip(y_pred_proba.iloc[:, 1], 0.001, 0.999)
    y_true[y_true == 0] = -1

    return np.clip(1 - y_true * np.log(probabilities / (1 - probabilities)), a_min=0, a_max=None)

report = explain_predictions_best_worst(pipeline=pipeline, input_features=X, y_true=y,
                                        include_shap_values=True, num_to_explain=5, metric=hinge_loss)

print(report)

#Single prediction as a dataframe

In [ ]:
single_prediction_report = explain_predictions(pipeline=pipeline, input_features=X, indices_to_explain=[3],
                                              y=y, top_k_features=6, include_shap_values=True,
                                              output_format="dataframe")
single_prediction_report

#Best and worst predictions as a dataframe

In [ ]:
report = explain_predictions_best_worst(pipeline=pipeline, input_features=X, y_true=y,
                                        num_to_explain=1, top_k_features=6,
                                        include_shap_values=True, output_format="dataframe")
report

#Code by Homayoon Khadivi https://www.kaggle.com/homayoonkhadivi/creative-automation-ml-evalml-model-understanding/comments